In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html, etree
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.text, "html.parser")

###### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
# Use Beautiful soup to find the table
post_table = soup.find("table",class_="wikitable sortable")
# Iterate through soup object and extract text from table
table_list = []
for rows in post_table.find_all('tr'):
    if rows.find_all('th'):
        postal_code = rows.find_all('th')[0].get_text()
        borough = rows.find_all('th')[1].get_text()
        neighborhood = rows.find_all('th')[2].get_text()[:-1]
    else:
        if rows.find_all('td')[1].a:
            postal_code = rows.find_all('td')[0].get_text()
            borough = rows.find_all('td')[1].get_text()
            neighborhood = rows.find_all('td')[2].get_text()[:-1]
        else:
            continue
    table_list.append([postal_code, borough, neighborhood])

In [6]:
# Create df from list
df = pd.DataFrame(table_list)
# Assign first row as header
df.columns = df.iloc[0]
# Create dataframe without first row
df = df[1:]
df.head(10)

,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Rouge
10,M1B,Scarborough,Malvern


In [7]:
# Rename column headers
df = df.rename(columns={'Postcode':'PostalCode', 'Borough':'Borough', 'Neighbourhood':'Neighborhood'})

###### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [8]:
# Groupby PostalCode and apply lambda function to combine neighborhoods together
grouped_df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
grouped_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
# Find out how many rows have 'Neighborhood' = 'Not assigned'
grouped_df[grouped_df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
83,M7A,Queen's Park,Not assigned


In [10]:
# Iterate through DF to assign Borough to Neighborhood
for i, row in grouped_df.iterrows():
    if grouped_df['Neighborhood'][i] == 'Not assigned':
        grouped_df['Neighborhood'][i] = grouped_df['Borough'][i]

In [11]:
grouped_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


###### Add longitude and latitude coordinates using geocoder

In [12]:
import geocoder

In [13]:
geocoder.google('Mountain View, CA')

<[OVER_QUERY_LIMIT] Google - Geocode [empty]>

###### Geocoder is not working. Using CSV file

In [14]:
file = '/Users/scott/Projects/Coursera/Geospatial_Coordinates.csv'

In [15]:
coord_df = pd.read_csv(file)

In [16]:
# Rename Postal Code column to match other df
coord_df = coord_df.rename(columns={'Postal Code': 'PostalCode'})

In [17]:
final_df = grouped_df.merge(coord_df, on='PostalCode', how='left')

In [18]:
final_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
